# Data Augmentation for Text Data


In [ ]:
%pip install --upgrade --user --quiet bigframes

In [ ]:
import sys

if "google.colab" in sys.modules:
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
from bigframes.ml.llm import GeminiTextGenerator
import bigframes.pandas as bpd

In [ ]:
PROJECT_ID = "caramel-gate-437719-q6"  # @param {type:"string"}
LOCATION = "US"  # @param {type:"string"}

bpd.options.bigquery.project = PROJECT_ID
bpd.options.bigquery.location = LOCATION

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

claims_df = pd.read_csv("/content/claims.csv", usecols=["claim", "claim_label"])

claims_df = claims_df.rename(columns={"claim_label": "label"})

print(claims_df.head())

In [ ]:
model = GeminiTextGenerator()

In [ ]:
n_replacement_words = 3  # Adjust this number based on your requirements

claims_df["synonym_prompt"] = (
    f"Replace {n_replacement_words} words from the input text with synonyms, "
    + "keeping the overall meaning as close to the original text as possible. "
    + "Only provide the synonymized text, with no additional explanation. "
    + "Preserve the original formatting.\n\nInput text: "
    + claims_df["claim"]
)

claims_df["claim_with_synonyms"] = model.predict(claims_df["synonym_prompt"]).ml_generate_text_llm_result

original_claims = claims_df[["claim", "label"]]

synonym_augmented_claims = claims_df[["claim_with_synonyms", "label"]].rename(columns={"claim_with_synonyms": "claim"})

augmented_claims_df = pd.concat([original_claims, synonym_augmented_claims], ignore_index=True)
augmented_claims_df = augmented_claims_df.dropna(subset=["claim"]).reset_index(drop=True)

In [ ]:
# Save the augmented DataFrame to a CSV file
augmented_claims_df.to_csv("/content/augmented_claims.csv", index=False)